In [1]:
import scipy.io as sio
import numpy as np
import pandas as pd
from pyspark.sql.functions import *

In [2]:
def import_file():       
    all_data = pd.DataFrame()
    path = "E:\\Cerin_big-data\\"
    #files = ["Test-Jan-01-Jan-05.xlsx", "Test-Jul-01-Jul-05.xlsx"]
    files = ["CERIn-Big-Data-2018-01-01-to-2018-07-01-Values.xlsx"]
    for f in files:
        full_path = path + f
        print(full_path)
        column_list = {'Phase A Current','Phase B Current','Phase C Current','Current Total','Power Factor',
                      'Frequency','Apparent Power','Active Power','Reactive Power','Power Production',
                       'Phase AB Voltage','Phase BC Voltage','Phase CA Voltage'}
        converters = {col: float for col in column_list}
        na_values=['I/O Timeout','Invalid Data']
        df = pd.read_excel(full_path,header=0, na_values=na_values, thousands=".", decimal=",")
        all_data = all_data.append(df,ignore_index=True)
    
    return all_data

In [3]:
def show_info(column, percentil, last_size , current_size):
    print("Column: {:>7} \t Percentile: {:>18} \t Bef/Actual/Del: {:>8} / {:>8} / {:>5}"
        .format(column, percentil, last_size, current_size, last_size - current_size))

In [4]:
def filter_data(dataset):
    last_size = len(dataset)
    colums = [k for k in list(dataset) if 'data' not in k.lower()]
    for column in colums:
        #dataset = dataset[pd.to_numeric(dataset[column], errors='coerce').notnull()]
        percentil = np.nanpercentile(dataset[column],1, interpolation='lower')
        
        ################# GOSTARIA DE UMA ATENÇÃO MAIOR AQUI, POIS NÃO SEI SE ESTÁ CORRETO ESSE TIPO DE 
        ################# VERIFICAÇÃO "> QUE".
        dataset = dataset[dataset[column] > percentil]
        
        current_size = len(dataset[column])
        show_info(column, percentil, last_size, current_size)        
        last_size = current_size
    return dataset	

In [5]:
def write_to_csv():
    path = "E://Cerin_big-data//dataframe3.csv"
    df.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").option("thousands", ".").option("decimal", ",").save(path)

In [6]:
data = import_file()
data.info()
data.head()

E:\Cerin_big-data\CERIn-Big-Data-2018-01-01-to-2018-07-01-Values.xlsx
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521401 entries, 0 to 521400
Data columns (total 14 columns):
Data                521401 non-null datetime64[ns]
Phase A Current     506599 non-null float64
Phase B Current     506599 non-null float64
Phase C Current     506599 non-null float64
Current Total       506599 non-null float64
Power Factor        240634 non-null float64
Frequency           506599 non-null float64
Apparent Power      240809 non-null float64
Active Power        506599 non-null float64
Reactive Power      240809 non-null float64
Power Production    506615 non-null float64
Phase AB Voltage    501435 non-null float64
Phase BC Voltage    501435 non-null float64
Phase CA Voltage    501435 non-null float64
dtypes: datetime64[ns](1), float64(13)
memory usage: 55.7 MB
Column: Phase A Current 	 Percentile:                0.0 	 Before/Current/Removed:   521401 /   423509 / 97892
Column: Phase B Current 

,Data,Phase A Current,Phase B Current,Phase C Current,Current Total,Power Factor,Frequency,Apparent Power,Active Power,Reactive Power,Power Production,Phase AB Voltage,Phase BC Voltage,Phase CA Voltage
4574,2018-01-02 14:07:00,3.335057,3.295057,3.335057,9.207820,-99.927567,59.981071,2005.604004,2005.488525,21.497658,29878138.0,388.025391,387.982635,387.053345
4575,2018-01-02 14:07:30,3.325196,3.285196,3.325196,8.901662,-99.924942,59.981052,1889.859009,1889.758179,19.510870,29878188.0,388.078949,387.959564,387.010468
4576,2018-01-02 14:08:00,3.315334,3.275334,3.315334,8.595504,-99.922318,59.981030,1762.594360,1762.482056,19.000000,29878202.0,388.132507,387.936493,386.967621
4577,2018-01-02 14:08:30,3.305472,3.265472,3.305472,8.289346,-99.919693,59.981010,1631.342407,1631.209595,19.000000,29878202.0,388.186066,387.913422,386.924774
4578,2018-01-02 14:09:00,3.295610,3.255610,3.295610,7.983187,-99.917068,59.980988,1518.218140,1518.077759,19.000000,29878202.0,388.131714,387.890350,386.881897


In [ ]:
filtered = filter_data(data)
filtered.info()
filtered.head()

In [7]:
df = spark.createDataFrame(filtered)

In [28]:
df = df.select(df['*'], weekofyear('Data').alias('WeekOfYear'), date_format('Data', 'u').alias('DayOfWeek'))

In [29]:
df.schema.fields

[StructField(Data,TimestampType,true),
 StructField(Phase A Current,DoubleType,true),
 StructField(Phase B Current,DoubleType,true),
 StructField(Phase C Current,DoubleType,true),
 StructField(Current Total,DoubleType,true),
 StructField(Power Factor,DoubleType,true),
 StructField(Frequency,DoubleType,true),
 StructField(Apparent Power,DoubleType,true),
 StructField(Active Power,DoubleType,true),
 StructField(Reactive Power,DoubleType,true),
 StructField(Power Production,DoubleType,true),
 StructField(Phase AB Voltage,DoubleType,true),
 StructField(Phase BC Voltage,DoubleType,true),
 StructField(Phase CA Voltage,DoubleType,true),
 StructField(WeekOfYear,IntegerType,true),
 StructField(DayOfWeek,StringType,true)]

In [74]:
report = df.select('WeekOfYear','DayOfWeek','Data','Power Production').groupBy("WeekOfYear",'DayOfWeek').agg(min("Data").alias('Start Date'), max("Data").alias('End Date'), min("Power Production").alias('Init PP'), max("Power Production").alias('Final PP'), (max("Power Production") -  min("Power Production")).alias('Total PP')).orderBy(asc("WeekOfYear"),asc("DayOfWeek"))
report.show()

+----------+---------+-------------------+-------------------+-----------+-----------+--------+
|WeekOfYear|DayOfWeek|         Start Date|           End Date|    Init PP|   Final PP|Total PP|
+----------+---------+-------------------+-------------------+-----------+-----------+--------+
|         1|        2|2018-01-02 14:07:00|2018-01-02 19:27:30|2.9878138E7|2.9893798E7| 15660.0|
|         1|        3|2018-01-03 07:28:30|2018-01-03 19:16:30|2.9893798E7|  2.99297E7| 35902.0|
|         1|        4|2018-01-04 06:34:00|2018-01-04 19:37:00|  2.99297E7|2.9980302E7| 50602.0|
|         1|        5|2018-01-05 06:47:00|2018-01-05 19:21:30|  2.99804E7|3.0026302E7| 45902.0|
|         1|        6|2018-01-06 07:03:00|2018-01-06 19:34:30|  3.00264E7|3.0059502E7| 33102.0|
|         1|        7|2018-01-07 07:04:30|2018-01-07 18:29:30|3.0059502E7|3.0089102E7| 29600.0|
|         2|        1|2018-01-08 06:40:30|2018-01-08 19:10:30|3.0089102E7|3.0117502E7| 28400.0|
|         2|        2|2018-01-09 06:38:0

In [12]:
catalog_name = "ElectricalValues"
spark.catalog.dropTempView(catalog_name)
df.createOrReplaceTempView(catalog_name)
spark.catalog.listTables()

[Table(name='electricalvalues', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [ ]:
write_to_csv()